## First Client Creation
The first time you create a client is kind of annoying and must be done with a username and password \
Copy secrets.sample into secrets.json and enter your username/password into that json file \
Then you should be able to run the next block in its entirety to generate a local cached client (`~/.tapis3/portals.json`) \
And in the future whenever you use TAPIS you can authenticate with this cached client instead of using username/password



In [ ]:
import json
from tapipy.tapis import Tapis
import os

with open("secrets.json", 'r') as openfile:
  secrets = json.load(openfile)

t = Tapis(base_url='https://portals.tapis.io', username=secrets['username'], password=secrets['password'])
t.get_tokens()
new_client = t.authenticator.create_client(client_id='new_user_client')

t2 = Tapis(base_url=t.base_url,
        username=secrets['username'],
        password=secrets['password'],
        client_id=new_client.client_id,
        client_key=new_client.client_key)
t2.get_tokens()

# You should now be able to run refresh_tokens with no errors. The t2 object will refresh when needed.
t2.refresh_tokens()

client_cache = {
    "base_url": t2.base_url,
    "tenant_id": t2.tenant_id,
    "client_id": t2.client_id,
    "client_key": t2.client_key,
    "access_token": t2.access_token.access_token,
    "refresh_token": t2.refresh_token.refresh_token,
    "expires_at": t2.access_token.expires_at.strftime("%Y-%m-%dT%H:%M:%S")
}
os.mkdir(os.path.expanduser("~/.tapis3"))
with open(os.path.expanduser('~/.tapis3/portals.json'), 'w') as f:
    json.dump(client_cache, f)

## Starting from the Cached Client
Ok, now we can use that cached client to create our TAPIS python object and deploy systems/apps/jobs etc.

In [11]:
import json
from tapipy.tapis import Tapis
import os

with open(os.path.expanduser('~/.tapis3/portals.json'), 'r') as openfile:
  data = json.load(openfile)



t = Tapis(base_url=data["base_url"],
           tenant_id=data["tenant_id"],
           access_token=data["access_token"],
           refresh_token=data["refresh_token"],
           client_id=data["client_id"],
           client_key=data["client_key"],
           verify=True)
t.get_tokens()


## Some Useful Commands

In [12]:
t.systems.getSystems(listType='ALL')
t.jobs.getJobList()
#t.jobs.getJob(jobUuid='')

[
 appId: jupyter-lab-hpc-ls6
 appVersion: 1.1.0
 archiveSystemId: cloud.data
 created: 2024-04-22T18:32:43.357014Z
 ended: None
 execSystemId: ls6
 lastUpdated: 2024-04-22T18:50:42.715145Z
 name: jupyter-lab-hpc-ls6-1.1.0_2024-04-22T18:32:31
 owner: urrutia
 remoteStarted: 2024-04-22T18:33:23.089482Z
 status: RUNNING
 tenant: portals
 uuid: bb7f08b4-698e-4d33-9460-a69d64d8236b-007]

In [19]:
# deploy new app
with open('new_app.json', 'r') as openfile:
    new_app = json.load(openfile)
t.apps.createAppVersion(**new_app)
# update existing app
#t.apps.patchApp(appId=new_app['id'],appVersion=new_app['version'],jobAttributes=new_app['jobAttributes'])


url: http://portals.tapis.io/v3/apps/urrutia-Autodock-Vina

In [ ]:
#submit a job
with open('job.json', 'r') as openfile:
    job = json.load(openfile)
submitted = t.jobs.submitJob(**job)